Training parameters: Working ion, crystal system, Spacegroup numbr
Output: max delta volume, volumetric capacity, gravimetric capacity

In [82]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [9]:
bat_data = pd.read_csv('TrainingData.csv')

In [23]:
bat_data

,Battery ID,Working Ion,Crystal System,Spacegroup Number,Gravimetric Capacity (units),Volumetric Capacity,Max Delta Volume,mean_Number,mean_MendeleevNumber,mean_AtomicWeight,...,dev_NValance.1,dev_NsUnfilled.1,dev_NpUnfilled.1,dev_NdUnfilled.1,dev_NfUnfilled.1,dev_NUnfilled.1,dev_GSvolume_pa.1,dev_GSbandgap.1,dev_GSmagmom.1,dev_SpaceGroupNumber.1
0,mp-504791_Li,Li,Orthorombic,19,50.228609,269.262771,0.028564,20.636364,80.090909,47.877178,...,4.055556,0.152778,0.791667,1.666667,0.00,1.166667,3.871508,0.248264,0.000000,81.791667
1,mp-763480_Li,Li,Monoclinic,14,39.674483,154.215698,0.017562,16.500000,82.333333,37.144083,...,2.725762,0.099723,0.581717,1.130194,0.00,0.880886,4.449100,0.432133,0.000000,58.204986
2,mp-1176966_Li,Li,Trigonal,165,181.943205,651.708695,0.061034,14.350000,83.500000,31.582317,...,1.918367,0.090703,0.464853,0.795918,0.00,0.727891,4.719358,0.501134,0.000000,45.969388
3,mvc-5592_Li,Li,Triclinic,2,73.484217,329.726946,0.021096,16.000000,82.600000,35.778332,...,2.429752,0.165289,0.661157,0.991736,0.00,0.842975,4.785895,0.483471,0.000000,65.223140
4,mp-763566_Li,Li,Monoclinic,4,73.484217,306.081406,0.025349,16.000000,82.600000,35.778332,...,2.429752,0.165289,0.661157,0.991736,0.00,0.842975,4.785895,0.483471,0.000000,65.223140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4396,mp-1180276_Mg,Mg,Cubic,215,92.144929,461.340369,0.069416,30.750000,74.250000,69.677437,...,3.950617,0.444444,0.987654,1.382716,0.00,1.111111,6.794568,0.394568,0.000000,101.530864
4397,mp-1042619_Mg,Mg,Cubic,204,60.789787,387.828816,0.006392,20.157895,81.894737,45.130042,...,3.195000,0.255000,0.800000,0.000000,0.00,0.740000,8.140500,0.000000,0.000000,80.220000
4398,mp-1046932_Mg,Mg,Triclinic,1,135.921407,914.229317,0.058047,24.100000,72.900000,55.509350,...,4.181474,0.000000,1.156900,0.574669,0.00,0.793951,11.118866,0.000000,0.171077,99.629490
4399,mp-1041103_Mg,Mg,Triclinic,1,112.348870,735.558063,0.019274,22.111111,72.777778,50.316302,...,1.340000,0.320000,0.960000,1.600000,0.54,1.380000,5.830000,0.000000,0.000000,95.760000


In [27]:
bat_data['Working Ion'].unique()

array(['Li', 'Ca', 'Cs', 'Rb', 'K', 'Y', 'Na', 'Al', 'Zn', 'Mg'],
      dtype=object)

1 Li
2 Ca
3 Cs
4 Rb
5 K
6 Y
7 Na
8 Al
9 Zn
0 Mg

In [29]:
bat_data['Crystal System'].unique()

array(['Orthorombic', 'Monoclinic', 'Trigonal', 'Triclinic', 'Tetragonal',
       'Hexagonal', 'Cubic'], dtype=object)

1 - Orthorombic
2 - Monoclinic
3 - Trigonal
4 - Triclinic
5 - Tetragonal
6 - Hexagonal
7 - Cubic


creating dataframe for training inputs:

In [67]:
train_bat = bat_data[['Working Ion','Crystal System', 'Spacegroup Number', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume']]

In [68]:
train_bat = train_bat.replace('Li', 1)
train_bat = train_bat.replace('Ca', 2)
train_bat = train_bat.replace('Cs', 3)
train_bat = train_bat.replace('Rb', 4)
train_bat = train_bat.replace('K', 5)
train_bat = train_bat.replace('Y', 6)
train_bat = train_bat.replace('Na', 7)
train_bat = train_bat.replace('Al', 8)
train_bat = train_bat.replace('Zn', 9)
train_bat = train_bat.replace('Mg', 0)

In [69]:
train_bat

,Working Ion,Crystal System,Spacegroup Number,Gravimetric Capacity (units),Volumetric Capacity,Max Delta Volume
0,1,Orthorombic,19,50.228609,269.262771,0.028564
1,1,Monoclinic,14,39.674483,154.215698,0.017562
2,1,Trigonal,165,181.943205,651.708695,0.061034
3,1,Triclinic,2,73.484217,329.726946,0.021096
4,1,Monoclinic,4,73.484217,306.081406,0.025349
...,...,...,...,...,...,...
4396,0,Cubic,215,92.144929,461.340369,0.069416
4397,0,Cubic,204,60.789787,387.828816,0.006392
4398,0,Triclinic,1,135.921407,914.229317,0.058047
4399,0,Triclinic,1,112.348870,735.558063,0.019274


In [70]:
train_bat = train_bat.replace('Orthorombic', 1)
train_bat = train_bat.replace('Monoclinic', 2)
train_bat = train_bat.replace('Trigonal', 3)
train_bat = train_bat.replace('Triclinic', 4)
train_bat = train_bat.replace('Tetragonal', 5)
train_bat = train_bat.replace('Hexagonal', 6)
train_bat = train_bat.replace('Cubic', 7)

In [71]:
train_bat

,Working Ion,Crystal System,Spacegroup Number,Gravimetric Capacity (units),Volumetric Capacity,Max Delta Volume
0,1,1,19,50.228609,269.262771,0.028564
1,1,2,14,39.674483,154.215698,0.017562
2,1,3,165,181.943205,651.708695,0.061034
3,1,4,2,73.484217,329.726946,0.021096
4,1,2,4,73.484217,306.081406,0.025349
...,...,...,...,...,...,...
4396,0,7,215,92.144929,461.340369,0.069416
4397,0,7,204,60.789787,387.828816,0.006392
4398,0,4,1,135.921407,914.229317,0.058047
4399,0,4,1,112.348870,735.558063,0.019274


In [72]:
np_train_bat = np.array(train_bat)

In [73]:
np_train_bat = np_train_bat.T

In [74]:
torch.tensor(np_train_bat)

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0000e+00, 2.0000e+00, 3.0000e+00,  ..., 4.0000e+00, 4.0000e+00,
         7.0000e+00],
        [1.9000e+01, 1.4000e+01, 1.6500e+02,  ..., 1.0000e+00, 1.0000e+00,
         2.3000e+02],
        [5.0229e+01, 3.9674e+01, 1.8194e+02,  ..., 1.3592e+02, 1.1235e+02,
         1.8907e+02],
        [2.6926e+02, 1.5422e+02, 6.5171e+02,  ..., 9.1423e+02, 7.3556e+02,
         1.1220e+03],
        [2.8564e-02, 1.7562e-02, 6.1034e-02,  ..., 5.8047e-02, 1.9274e-02,
         3.1063e-03]], dtype=torch.float64)

# Neural Network

In [84]:
dtype = torch.float
device = torch.device("cpu")

In [85]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 50, 1000, 100, 10

In [86]:
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [88]:
# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

In [89]:
loss_fn = torch.nn.MSELoss(reduction='sum')

In [90]:
learning_rate = 1e-4
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 1.431857943534851
199 0.0193927139043808
299 0.0008012093021534383
399 4.713802263722755e-05
499 3.085646540057496e-06


In [92]:

from torch import nn

# define model architecture
model = nn.Sequential(
    nn.Linear(6, 4),
    nn.ReLU(),
    nn.Linear(4, 2),
    nn.ReLU(),
    nn.Linear(2, 1),
    nn.Sigmoid()
)

# print model architecture
print(model)

Sequential(
  (0): Linear(in_features=6, out_features=4, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): ReLU()
  (4): Linear(in_features=2, out_features=1, bias=True)
  (5): Sigmoid()
)
